In [ ]:
import pandas as pd
import numpy as np
from transformers import pipeline

In [ ]:
def read_dataset(file):
    data = pd.read_csv(file, delimiter='\t', header = None).drop(columns=[0])
    cols = ['id','label','statement','subjects','speaker','speaker_job_title','state_info','party_affiliation',
        'barely_true','false','half_true','mostly_true','pants_fire','context','justification']
    data.columns = cols

#     mapping = {
#         "true": 0,
#         "mostly-true": 1,
#         "half-true": 2,
#         "barely-true": 3,
#         "false": 4,
#         "pants-fire": 5
#     }

#     data['label'] = data['label'].map(mapping)

    return data

In [ ]:
train = read_dataset('train2.tsv').dropna()
valid = read_dataset("val2.tsv").dropna()
test = read_dataset("test2.tsv").dropna()

In [ ]:
train.head(1)

,id,label,statement,subjects,speaker,speaker_job_title,state_info,party_affiliation,barely_true,false,half_true,mostly_true,pants_fire,context,justification
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer,That's a premise that he fails to back up. Ann...


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

X_train = train[["barely_true", "false", "half_true", "mostly_true", "pants_fire"]]
y_train = train[["label"]]

X_test = train[["barely_true", "false", "half_true", "mostly_true", "pants_fire"]]
y_test = train[["label"]]

clf = LogisticRegression(random_state=0, max_iter=1000).fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

In [ ]:
train["label"].unique()

array(['false', 'half-true', 'mostly-true', 'true', 'barely-true',
       'pants-fire'], dtype=object)

In [ ]:
test['justification'][0]

'Meantime, engineering experts agree the wall would most likely take years to complete. Keep in mind, too, it took more than six years to build roughly 700 miles of fence and barriers along the roughly 2,000-mile U. S. -Mexico border.'

In [ ]:
test['statement'][0]

'Building a wall on the U.S.-Mexico border will take literally years.'

In [ ]:
# train.dropna()
# from transformers import BertTokenizer, BertModel
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader, TensorDataset
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# tokenizer = BertTokenizer.from_pretrained('google/mobilebert-uncased')
# model = BertModel.from_pretrained('google/mobilebert-uncased')
# statements = train['statement'].astype(str).tolist()
# encoded_inputs = tokenizer(statements, padding=True, truncation=True, return_tensors='pt')

In [ ]:
# statements

In [ ]:
# with torch.no_grad():
#     outputs = model(**encoded_inputs)
#     embeddings = outputs.last_hidden_state.mean(dim=1)

# train_embeddings_tensor = embeddings
# train_labels_tensor = torch.tensor(train['label'])

# train_embeddings, val_embeddings, train_labels, val_labels = train_test_split(train_embeddings_tensor,
#                                                                               train_labels_tensor,
#                                                                               test_size=0.2,
#                                                                               random_state=42)
# class CustomBERTClassifier(nn.Module):
#     def __init__(self, input_dim, hidden_dim, output_dim):
#         super().__init__()
#         self.fc1 = nn.Linear(input_dim, hidden_dim)
#         self.fc2 = nn.Linear(hidden_dim, output_dim)
#         self.dropout = nn.Dropout(0.2)

#     def forward(self, x):
#         x = self.dropout(x)
#         x = nn.functional.relu(self.fc1(x))
#         x = self.fc2(x)
#         return x

# model = CustomBERTClassifier(input_dim=768, hidden_dim=128, output_dim=6)

# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=1e-5)

# epochs = 5
# batch_size = 32

# for epoch in range(epochs):
#     running_loss = 0.0
#     for i in range(0, len(train_embeddings), batch_size):
#         optimizer.zero_grad()
#         outputs = model(train_embeddings[i:i+batch_size])
#         loss = criterion(outputs, train_labels[i:i+batch_size])
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()

#     print(f"Epoch {epoch+1}/{epochs} - Loss: {running_loss}")

# model.eval()
# with torch.no_grad():
#     val_outputs = model(val_embeddings)
#     val_preds = val_outputs.argmax(dim=1)
#     val_acc = accuracy_score(val_labels.numpy(), val_preds.numpy())
#     val_precision = precision_score(val_labels.numpy(), val_preds.numpy(), average='weighted')
#     val_recall = recall_score(val_labels.numpy(), val_preds.numpy(), average='weighted')
#     val_f1 = f1_score(val_labels.numpy(), val_preds.numpy(), average='weighted')
#     val_conf_matrix = confusion_matrix(val_labels.numpy(), val_preds.numpy())

# print(f"Validation Accuracy: {val_acc}")
# print(f"Validation Precision: {val_precision}")
# print(f"Validation Recall: {val_recall}")
# print(f"Validation F1 Score: {val_f1}")
# print("Validation Confusion Matrix:")
# print(val_conf_matrix)

In [ ]:
# from sklearn.preprocessing import LabelEncoder

# # Assuming 'train_data' contains the dataset with 'statement' (textual statements) and 'label' (string labels) columns
# labels = train['label']

# # Initialize LabelEncoder
# label_encoder = LabelEncoder()

# # Fit label encoder and transform string labels to numerical values
# numerical_labels = label_encoder.fit_transform(labels)

# # Replace string labels in the DataFrame with numerical labels
# train['label'] = numerical_labels

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense


max_words = 1000
max_len = 100

label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train['label'])
test_labels = label_encoder.transform(test['label'])
valid_labels = label_encoder.transform(valid['label'])

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train['statement'])
train_sequences = tokenizer.texts_to_sequences(train['statement'])
test_sequences = tokenizer.texts_to_sequences(test['statement'])
valid_sequences = tokenizer.texts_to_sequences(valid['statement'])
word_index = tokenizer.word_index

train_data = pad_sequences(train_sequences, maxlen=max_len)
test_data = pad_sequences(test_sequences, maxlen=max_len)
valid_data = pad_sequences(valid_sequences, maxlen=max_len)

model = Sequential()
model.add(Embedding(max_words, 32, input_length=max_len))
model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))  # Softmax for multi-class output

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_data, train_labels, epochs=10, batch_size=32, validation_data=(valid_data, valid_labels))

loss, accuracy = model.evaluate(test_data, test_labels)
print(f'Test Accuracy: {accuracy}')

Epoch 1/10
209/209 [==============================] - 1s 4ms/step - loss: 1.7457 - accuracy: 0.2098 - val_loss: 1.7484 - val_accuracy: 0.2145
Epoch 2/10
209/209 [==============================] - 1s 4ms/step - loss: 1.7224 - accuracy: 0.2666 - val_loss: 1.7402 - val_accuracy: 0.2145
Epoch 3/10
209/209 [==============================] - 1s 4ms/step - loss: 1.6495 - accuracy: 0.3210 - val_loss: 1.7374 - val_accuracy: 0.2179
Epoch 4/10
209/209 [==============================] - 1s 4ms/step - loss: 1.5059 - accuracy: 0.4049 - val_loss: 1.7784 - val_accuracy: 0.2238
Epoch 5/10
209/209 [==============================] - 1s 4ms/step - loss: 1.3237 - accuracy: 0.5038 - val_loss: 1.8611 - val_accuracy: 0.2261
Epoch 6/10
209/209 [==============================] - 1s 5ms/step - loss: 1.1407 - accuracy: 0.5758 - val_loss: 2.0060 - val_accuracy: 0.2296
Epoch 7/10
209/209 [==============================] - 1s 4ms/step - loss: 0.9670 - accuracy: 0.6506 - val_loss: 2.1494 - val_accuracy: 0.2168
Epoch 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, Flatten, Dense, Input, Concatenate

max_words = 1000
max_len = 100

label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train['label'])
test_labels = label_encoder.transform(test['label'])
valid_labels = label_encoder.transform(valid['label'])

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train['statement'])
train_sequences = tokenizer.texts_to_sequences(train['statement'])
test_sequences = tokenizer.texts_to_sequences(test['statement'])
valid_sequences = tokenizer.texts_to_sequences(valid['statement'])
word_index = tokenizer.word_index

train_data_text = pad_sequences(train_sequences, maxlen=max_len)
test_data_text = pad_sequences(test_sequences, maxlen=max_len)
valid_data_text = pad_sequences(valid_sequences, maxlen=max_len)

count_features = ['barely_true', 'false', 'half_true', 'mostly_true', 'pants_fire']
scaler = StandardScaler()
train_counts = scaler.fit_transform(train[count_features])
test_counts = scaler.transform(test[count_features])
valid_counts = scaler.transform(valid[count_features])

text_input = Input(shape=(max_len,))
count_input = Input(shape=(len(count_features),))

text_model = Embedding(max_words, 32, input_length=max_len)(text_input)
text_model = Flatten()(text_model)

count_model = Dense(16, activation='relu')(count_input)

combined = Concatenate()([text_model, count_model])
combined = Dense(16, activation='relu')(combined)
output = Dense(len(label_encoder.classes_), activation='softmax')(combined)

model = Model(inputs=[text_input, count_input], outputs=output)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit([train_data_text, train_counts], train_labels, epochs=10, batch_size=32,
                    validation_data=([valid_data_text, valid_counts], valid_labels))

loss, accuracy = model.evaluate([test_data_text, test_counts], test_labels)
print(f'Test Accuracy: {accuracy}')

Epoch 1/10
209/209 [==============================] - 1s 4ms/step - loss: 1.7387 - accuracy: 0.2150 - val_loss: 1.7392 - val_accuracy: 0.2203
Epoch 2/10
209/209 [==============================] - 1s 3ms/step - loss: 1.6997 - accuracy: 0.2714 - val_loss: 1.6996 - val_accuracy: 0.2529
Epoch 3/10
209/209 [==============================] - 1s 3ms/step - loss: 1.6396 - accuracy: 0.3186 - val_loss: 1.6906 - val_accuracy: 0.2587
Epoch 4/10
209/209 [==============================] - 1s 3ms/step - loss: 1.5475 - accuracy: 0.3704 - val_loss: 1.7128 - val_accuracy: 0.2622
Epoch 5/10
209/209 [==============================] - 1s 3ms/step - loss: 1.4030 - accuracy: 0.4416 - val_loss: 1.7796 - val_accuracy: 0.2331
Epoch 6/10
209/209 [==============================] - 1s 4ms/step - loss: 1.2490 - accuracy: 0.5134 - val_loss: 1.8783 - val_accuracy: 0.2424
Epoch 7/10
209/209 [==============================] - 1s 4ms/step - loss: 1.0953 - accuracy: 0.5870 - val_loss: 2.0190 - val_accuracy: 0.2354
Epoch 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input, concatenate, Dropout

max_words = 1000
max_len = 100

label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train['label'])
test_labels = label_encoder.transform(test['label'])
valid_labels = label_encoder.transform(valid['label'])

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train['statement'])
train_sequences = tokenizer.texts_to_sequences(train['statement'])
test_sequences = tokenizer.texts_to_sequences(test['statement'])
valid_sequences = tokenizer.texts_to_sequences(valid['statement'])
word_index = tokenizer.word_index

train_data_text = pad_sequences(train_sequences, maxlen=max_len)
test_data_text = pad_sequences(test_sequences, maxlen=max_len)
valid_data_text = pad_sequences(valid_sequences, maxlen=max_len)

count_features = ['barely_true', 'false', 'half_true', 'mostly_true', 'pants_fire']
scaler = StandardScaler()
train_counts = scaler.fit_transform(train[count_features])
test_counts = scaler.transform(test[count_features])
valid_counts = scaler.transform(valid[count_features])

text_input = Input(shape=(max_len,))
count_input = Input(shape=(len(count_features),))

text_model = Embedding(max_words, 32, input_length=max_len)(text_input)
text_model = LSTM(64, return_sequences=True)(text_model)
text_model = LSTM(64)(text_model)
text_model = Dropout(0.5)(text_model)

count_model = Dense(32, activation='relu')(count_input)
count_model = Dense(32, activation='relu')(count_model)
count_model = Dropout(0.5)(count_model)

combined = concatenate([text_model, count_model])
combined = Dense(32, activation='relu')(combined)
output = Dense(len(label_encoder.classes_), activation='softmax')(combined)

model = Model(inputs=[text_input, count_input], outputs=output)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit([train_data_text, train_counts], train_labels, epochs=10, batch_size=32,
                    validation_data=([valid_data_text, valid_counts], valid_labels))

loss, accuracy = model.evaluate([test_data_text, test_counts], test_labels)
print(f'Test Accuracy: {accuracy}')

Epoch 1/10
209/209 [==============================] - 23s 92ms/step - loss: 1.7446 - accuracy: 0.2129 - val_loss: 1.7238 - val_accuracy: 0.2203
Epoch 2/10
209/209 [==============================] - 18s 85ms/step - loss: 1.7040 - accuracy: 0.2446 - val_loss: 1.7000 - val_accuracy: 0.2622
Epoch 3/10
209/209 [==============================] - 18s 85ms/step - loss: 1.6618 - accuracy: 0.2687 - val_loss: 1.6913 - val_accuracy: 0.2692
Epoch 4/10
209/209 [==============================] - 18s 87ms/step - loss: 1.6264 - accuracy: 0.2888 - val_loss: 1.7008 - val_accuracy: 0.2564
Epoch 5/10
209/209 [==============================] - 18s 87ms/step - loss: 1.5854 - accuracy: 0.3207 - val_loss: 1.7165 - val_accuracy: 0.2576
Epoch 6/10
209/209 [==============================] - 18s 87ms/step - loss: 1.5371 - accuracy: 0.3543 - val_loss: 1.7465 - val_accuracy: 0.2389
Epoch 7/10
209/209 [==============================] - 18s 85ms/step - loss: 1.4915 - accuracy: 0.3776 - val_loss: 1.7381 - val_accuracy:

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

train_text = train['statement'] + ' ' + train['justification']
test_text = test['statement'] + ' ' + test['justification']
valid_text = valid['statement'] + ' ' + valid['justification']

vectorizer = CountVectorizer()

X_train = vectorizer.fit_transform(train_text)
y_train = train['label']

X_test = vectorizer.transform(test_text)
y_test = test['label']

X_valid = vectorizer.transform(valid_text)
y_valid = valid['label']

nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)

y_pred_test = nb_classifier.predict(X_test)
y_pred_valid = nb_classifier.predict(X_valid)

accuracy_test = accuracy_score(y_test, y_pred_test)
accuracy_valid = accuracy_score(y_valid, y_pred_valid)

print(f'Test Accuracy: {accuracy_test}')
print(f'Validation Accuracy: {accuracy_valid}')

print('\nTest Classification Report:')
print(classification_report(y_test, y_pred_test))

print('\nValidation Classification Report:')
print(classification_report(y_valid, y_pred_valid))

Test Accuracy: 0.22968197879858657
Validation Accuracy: 0.23193473193473194

Test Classification Report:
              precision    recall  f1-score   support

 barely-true       0.16      0.05      0.08       139
       false       0.18      0.15      0.16       167
   half-true       0.24      0.51      0.33       181
 mostly-true       0.25      0.22      0.23       173
  pants-fire       0.00      0.00      0.00        42
        true       0.25      0.22      0.24       147

    accuracy                           0.23       849
   macro avg       0.18      0.19      0.17       849
weighted avg       0.21      0.23      0.20       849


Validation Classification Report:
              precision    recall  f1-score   support

 barely-true       0.27      0.09      0.14       154
       false       0.23      0.21      0.22       174
   half-true       0.21      0.40      0.28       173
 mostly-true       0.29      0.32      0.30       173
  pants-fire       0.00      0.00      0.00   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from scipy.sparse import hstack
import numpy as np

train_text = train['statement'] + ' ' + train['justification']
test_text = test['statement'] + ' ' + test['justification']
valid_text = valid['statement'] + ' ' + valid['justification']

features = ['barely_true', 'false', 'half_true', 'mostly_true', 'pants_fire']
X_train_numeric = train[features].values
X_test_numeric = test[features].values
X_valid_numeric = valid[features].values

vectorizer = CountVectorizer()

X_train_text = vectorizer.fit_transform(train_text)

X_test_text = vectorizer.transform(test_text)
X_valid_text = vectorizer.transform(valid_text)

X_train_combined = hstack([X_train_text, X_train_numeric])
X_test_combined = hstack([X_test_text, X_test_numeric])
X_valid_combined = hstack([X_valid_text, X_valid_numeric])

nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_combined, train['label'])

y_pred_test = nb_classifier.predict(X_test_combined)
y_pred_valid = nb_classifier.predict(X_valid_combined)

accuracy_test = accuracy_score(test['label'], y_pred_test)
accuracy_valid = accuracy_score(valid['label'], y_pred_valid)

print(f'Test Accuracy: {accuracy_test}')
print(f'Validation Accuracy: {accuracy_valid}')

print('\nTest Classification Report:')
print(classification_report(test['label'], y_pred_test))

print('\nValidation Classification Report:')
print(classification_report(valid['label'], y_pred_valid))

Test Accuracy: 0.22850412249705537
Validation Accuracy: 0.26573426573426573

Test Classification Report:
              precision    recall  f1-score   support

 barely-true       0.16      0.24      0.19       139
       false       0.19      0.21      0.20       167
   half-true       0.29      0.35      0.32       181
 mostly-true       0.26      0.13      0.18       173
  pants-fire       0.33      0.07      0.12        42
        true       0.25      0.24      0.25       147

    accuracy                           0.23       849
   macro avg       0.25      0.21      0.21       849
weighted avg       0.24      0.23      0.22       849


Validation Classification Report:
              precision    recall  f1-score   support

 barely-true       0.21      0.25      0.23       154
       false       0.30      0.29      0.29       174
   half-true       0.28      0.38      0.32       173
 mostly-true       0.34      0.23      0.27       173
  pants-fire       0.33      0.10      0.15   

In [ ]:
# import pandas as pd
# import numpy as np
# import torch
# from transformers import BertTokenizer, BertForSequenceClassification, AdamW
# from sklearn.model_selection import train_test_split
# from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
# from sklearn.preprocessing import LabelEncoder
# from sklearn.metrics import accuracy_score, classification_report

# train_text = train['statement'] + ' ' + train['justification']
# test_text = test['statement'] + ' ' + test['justification']
# valid_text = valid['statement'] + ' ' + valid['justification']

# label_encoder = LabelEncoder()
# train['label'] = label_encoder.fit_transform(train['label'])
# test['label'] = label_encoder.transform(test['label'])
# valid['label'] = label_encoder.transform(valid['label'])

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# train_encodings = tokenizer(list(train_text), truncation=True, padding=True)
# test_encodings = tokenizer(list(test_text), truncation=True, padding=True)
# valid_encodings = tokenizer(list(valid_text), truncation=True, padding=True)

# train_inputs = torch.tensor(train_encodings['input_ids'])
# train_masks = torch.tensor(train_encodings['attention_mask'])
# train_labels = torch.tensor(train['label'].values)

# test_inputs = torch.tensor(test_encodings['input_ids'])
# test_masks = torch.tensor(test_encodings['attention_mask'])
# test_labels = torch.tensor(test['label'].values)

# valid_inputs = torch.tensor(valid_encodings['input_ids'])
# valid_masks = torch.tensor(valid_encodings['attention_mask'])
# valid_labels = torch.tensor(valid['label'].values)

# batch_size = 16

# train_data = TensorDataset(train_inputs, train_masks, train_labels)
# train_sampler = RandomSampler(train_data)
# train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# test_data = TensorDataset(test_inputs, test_masks, test_labels)
# test_sampler = SequentialSampler(test_data)
# test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

# valid_data = TensorDataset(valid_inputs, valid_masks, valid_labels)
# valid_sampler = SequentialSampler(valid_data)
# valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_size)

# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(np.unique(train['label'])))
# optimizer = AdamW(model.parameters(), lr=2e-5)

# epochs = 3

# for epoch in range(epochs):
#     model.train()
#     total_loss = 0
#     for batch in train_dataloader:
#         batch = tuple(t.to(torch.device('cpu')) for t in batch)
#         inputs, masks, labels = batch
#         optimizer.zero_grad()
#         outputs = model(inputs, attention_mask=masks, labels=labels)
#         loss = outputs.loss
#         total_loss += loss.item()
#         loss.backward()
#         optimizer.step()

#     avg_train_loss = total_loss / len(train_dataloader)

#     model.eval()
#     val_accuracy = 0
#     val_preds = []
#     val_labels = []
#     for batch in valid_dataloader:
#         batch = tuple(t.to(torch.device('cpu')) for t in batch)
#         inputs, masks, labels = batch
#         with torch.no_grad():
#             outputs = model(inputs, attention_mask=masks)
#         logits = outputs.logits
#         preds = torch.argmax(logits, dim=1).cpu().numpy()
#         val_preds.extend(preds)
#         val_labels.extend(labels.cpu().numpy())
#     val_accuracy = accuracy_score(val_labels, val_preds)

#     print(f'Epoch {epoch + 1}/{epochs}')
#     print(f'Training loss: {avg_train_loss}')
#     print(f'Validation accuracy: {val_accuracy}')

# test_preds = []
# test_labels = []
# for batch in test_dataloader:
#     batch = tuple(t.to(torch.device('cpu')) for t in batch)
#     inputs, masks, labels = batch
#     with torch.no_grad():
#         outputs = model(inputs, attention_mask=masks)
#     logits = outputs.logits
#     preds = torch.argmax(logits, dim=1).cpu().numpy()
#     test_preds.extend(preds)
#     test_labels.extend(labels.cpu().numpy())

# test_accuracy = accuracy_score(test_labels, test_preds)
# print(f'Test accuracy: {test_accuracy}')
# print('Test Classification Report:')
# print(classification_report(test_labels, test_preds))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# from transformers import BertTokenizer, BertForSequenceClassification
# import torch
# import torch.nn as nn
# from torch.utils.data import TensorDataset, DataLoader
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# # Load LIAR-PLUS dataset (load, preprocess, split into train/validation sets)

# # Assuming train_data and val_data contain 'statement' (textual statements) and 'label' (numerical labels) columns

# # Initialize BERT tokenizer and model
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)  # 6 classes for 6 label targets

# # Tokenize and encode textual statements and convert labels to tensors
# train_encodings = tokenizer(list(train['statement']), truncation=True, padding=True, return_tensors='pt')
# val_encodings = tokenizer(list(valid['statement']), truncation=True, padding=True, return_tensors='pt')

# train_labels = torch.tensor(train['label'])
# val_labels = torch.tensor(valid['label'])

# # Prepare TensorDatasets for DataLoader
# train_dataset = TensorDataset(train_encodings.input_ids, train_encodings.attention_mask, train_labels)
# val_dataset = TensorDataset(val_encodings.input_ids, val_encodings.attention_mask, val_labels)

# # Define DataLoader for training and validation sets
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=32)

# # Define optimizer and loss function
# optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
# criterion = nn.CrossEntropyLoss()

# # Training loop
# epochs = 3
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)

# for epoch in range(epochs):
#     model.train()
#     for batch in train_loader:
#         input_ids, attention_mask, labels = batch
#         optimizer.zero_grad()
#         input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss
#         loss.backward()
#         optimizer.step()

# # Evaluation
# model.eval()
# val_predictions = []
# val_targets = []

# for batch in val_loader:
#     input_ids, attention_mask, labels = batch
#     input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
#     with torch.no_grad():
#         outputs = model(input_ids, attention_mask=attention_mask)
#         logits = outputs.logits
#         predictions = logits.argmax(dim=1)
#         val_predictions.extend(predictions.cpu().numpy())
#         val_targets.extend(labels.cpu().numpy())

# # Calculate evaluation metrics
# accuracy = accuracy_score(val_targets, val_predictions)
# precision = precision_score(val_targets, val_predictions, average='weighted')
# recall = recall_score(val_targets, val_predictions, average='weighted')
# f1 = f1_score(val_targets, val_predictions, average='weighted')
# conf_matrix = confusion_matrix(val_targets, val_predictions)

# # Print evaluation metrics
# print(f'Accuracy: {accuracy}')
# print(f'Precision: {precision}')
# print(f'Recall: {recall}')
# print(f'F1-Score: {f1}')
# print('Confusion Matrix:')
# print(conf_matrix)

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

train_text = train['statement'] + ' ' + train['justification']
test_text = test['statement'] + ' ' + test['justification']
valid_text = valid['statement'] + ' ' + valid['justification']

train_labels = train['label']
test_labels = test['label']
valid_labels = valid['label']

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to generate BERT embeddings
def generate_bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    sentence_embedding = last_hidden_states[:, 0, :]
    return sentence_embedding.numpy()

# Generate BERT embeddings for train, test, and validation sets
train_embeddings = [generate_bert_embedding(text) for text in train_text]
test_embeddings = [generate_bert_embedding(text) for text in test_text]
valid_embeddings = [generate_bert_embedding(text) for text in valid_text]

# Convert embeddings to NumPy arrays
X_train = np.array([emb.squeeze() for emb in train_embeddings])
X_test = np.array([emb.squeeze() for emb in test_embeddings])
X_valid = np.array([emb.squeeze() for emb in valid_embeddings])

# Encode labels
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_labels)
y_test = label_encoder.transform(test_labels)
y_valid = label_encoder.transform(valid_labels)

from sklearn import svm
# Initialize Logistic Regression model
logreg = LogisticRegression(multi_class='multinomial', solver='lbfgs')
svm_model = svm.LinearSVC()

# Fit the Logistic Regression model
logreg.fit(X_train, y_train)
svm_model.fit(X_train, y_train)

# Predictions on test data
y_pred = logreg.predict(X_test)
y_pred2 = svm_model.predict(X_test)

# Calculate accuracy
accuracy1 = accuracy_score(y_test, y_pred)
accuracy2 = accuracy_score(y_test, y_pred2)
print(f'Test Accuracy Logistic Regression: {accuracy1}')
print(classification_report(y_test, y_pred))

print(f'Test Accuracy LinearSVM: {accuracy2}')
print(classification_report(y_test, y_pred2))

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Test Accuracy Logistic Regression: 0.24028268551236748
              precision    recall  f1-score   support

           0       0.15      0.10      0.12       139
           1       0.24      0.27      0.25       167
           2       0.24      0.26      0.25       181
           3       0.29      0.35      0.32       173
           4       0.17      0.10      0.12        42
           5       0.25      0.22      0.24       147

    accuracy                           0.24       849
   macro avg       0.22      0.22      0.22       849
weighted avg       0.23      0.24      0.23       849

Test Accuracy LinearSVM: 0.21554770318021202
              precision    recall  f1-score   support

           0       0.13      0.15      0.14       139
           1       0.25      0.22      0.23       167
           2       0.23      0.19      0.21       181
           3       0.29      0.38      0.33       173
           4       0.10      0.10      0.10        42
           5       0.18      0.1

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [ ]:
from sklearn import svm

svm_model = svm.LinearSVC()
svm_model.fit(X_train, y_train)

y_pred = svm_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {accuracy}')

print(classification_report(y_test, y_pred))

Test Accuracy: 0.22143698468786807
              precision    recall  f1-score   support

           0       0.13      0.12      0.13       139
           1       0.24      0.19      0.21       167
           2       0.23      0.22      0.22       181
           3       0.30      0.38      0.33       173
           4       0.06      0.05      0.05        42
           5       0.21      0.23      0.22       147

    accuracy                           0.22       849
   macro avg       0.19      0.20      0.19       849
weighted avg       0.22      0.22      0.22       849



/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=2, random_state = 77)
clf.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {accuracy}')

print(classification_report(y_test, y_pred))

Test Accuracy: 0.22143698468786807
              precision    recall  f1-score   support

           0       0.13      0.12      0.13       139
           1       0.24      0.19      0.21       167
           2       0.23      0.22      0.22       181
           3       0.30      0.38      0.33       173
           4       0.06      0.05      0.05        42
           5       0.21      0.23      0.22       147

    accuracy                           0.22       849
   macro avg       0.19      0.20      0.19       849
weighted avg       0.22      0.22      0.22       849

